In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import sys
sys.path.append('../scripts')

In [3]:
import os
# Disable weights and biases (if installed)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["WANDB_DISABLED"] = "true"

In [4]:
from pathlib import Path

from dataset import load_data, get_dataloader
from generative.run_experiment import get_training_args, get_trainer, get_tokenizer

/home/Florian.Borchert/miniconda3/envs/ellipses/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Training

In [5]:
import hydra
from hydra import compose, initialize

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path=Path('..'), job_name='foo', version_base='1.1')
config = compose(config_name='experiment.yaml')

In [6]:
training_args = get_training_args(config, report_to="none")
tokenizer = get_tokenizer(config)

/home/Florian.Borchert/miniconda3/envs/ellipses/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [7]:
base_path = Path('..')
train_df, val_df, test_df = load_data(base_path / config.data.cnf_tsv_path, base_path / config.data.controls_tsv_path)

In [8]:
# Ellipses
(~train_df.controls).sum(), (~val_df.controls).sum(), (~test_df.controls).sum()

(2317, 463, 488)

In [9]:
# Controls
(train_df.controls).sum(), (val_df.controls).sum(), (test_df.controls).sum()

(2239, 500, 529)

In [10]:
train_dataset, val_dataset, test_dataset = get_dataloader(train_df, val_df, test_df, tokenizer)

In [11]:
training_args.num_train_epochs = 10

In [12]:
trainer = get_trainer(config, tokenizer, training_args, train_dataset, val_dataset)

In [13]:
trainer.train()

/home/Florian.Borchert/miniconda3/envs/ellipses/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4556
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5700
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Exact Match,Google Bleu
1,0.168500,0.051593,0.757009,0.803461
2,0.072400,0.036552,0.808930,0.914712
3,0.045600,0.036677,0.821391,0.944792
4,0.033600,0.036598,0.831776,0.893739
5,0.021200,0.032676,0.859813,0.929487
6,0.017000,0.032514,0.865005,0.957693
7,0.013900,0.037077,0.877466,0.961684
8,0.010100,0.038767,0.873313,0.960636
9,0.004600,0.038103,0.874351,0.963395
10,0.004200,0.038547,0.878505,0.971691


***** Running Evaluation *****
  Num examples = 963
  Batch size = 8
***** Running Evaluation *****
  Num examples = 963
  Batch size = 8
***** Running Evaluation *****
  Num examples = 963
  Batch size = 8
***** Running Evaluation *****
  Num examples = 963
  Batch size = 8
***** Running Evaluation *****
  Num examples = 963
  Batch size = 8
***** Running Evaluation *****
  Num examples = 963
  Batch size = 8
***** Running Evaluation *****
  Num examples = 963
  Batch size = 8
***** Running Evaluation *****
  Num examples = 963
  Batch size = 8
***** Running Evaluation *****
  Num examples = 963
  Batch size = 8
***** Running Evaluation *****
  Num examples = 963
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=5700, training_loss=0.21733053085573933, metrics={'train_runtime': 3076.4843, 'train_samples_per_second': 14.809, 'train_steps_per_second': 1.853, 'total_flos': 9636508939419648.0, 'train_loss': 0.21733053085573933, 'epoch': 10.0})

In [15]:
eval_metrics = trainer.evaluate(val_dataset)
print(eval_metrics)

test_metrics = trainer.evaluate(test_dataset, metric_key_prefix='test')
print(test_metrics)

***** Running Evaluation *****
  Num examples = 963
  Batch size = 8


***** Running Evaluation *****
  Num examples = 1017
  Batch size = 8


{'eval_loss': 0.03854713588953018, 'eval_exact_match': 0.8785046728971962, 'eval_google_bleu': 0.9716914437064991, 'eval_runtime': 141.8926, 'eval_samples_per_second': 6.787, 'eval_steps_per_second': 0.853, 'epoch': 10.0}
{'test_loss': 0.039304446429014206, 'test_exact_match': 0.8790560471976401, 'test_google_bleu': 0.9637818113772455, 'test_runtime': 152.0005, 'test_samples_per_second': 6.691, 'test_steps_per_second': 0.842, 'epoch': 10.0}


# Error Analysis

In [16]:
from transformers import Text2TextGenerationPipeline
from evaluation import error_analysis, Metrics

In [17]:
pipeline = Text2TextGenerationPipeline(model=trainer.model, tokenizer=tokenizer, max_length=config.generation_max_length, device=0)

In [44]:
from evaluation import error_analysis, get_scores, encode_decode
from notebook_util import show_errors

ImportError: cannot import name 'encode_decode' from 'evaluation' (/mnt/nfs/home/Florian.Borchert/workspace/ggponc_ellipses/notebooks/../scripts/evaluation.py)

In [45]:
import pandas as pd

def calculate_errors(out, sample):
    gen_text = [o['generated_text'] for o in out]
    errors = error_analysis(gen_text, encode_decode(sample.full_resolution), encode_decode(sample.raw_sentence))
    errors = pd.concat([errors, sample[['file', 'sentence_id']].reset_index()], axis=1)
    return errors

In [20]:
%%time
out_valid = pipeline(list(val_df.raw_sentence))

CPU times: user 7min 41s, sys: 162 ms, total: 7min 41s
Wall time: 7min 41s


In [41]:
errors_valid = calculate_errors(out_valid, val_df)

In [46]:
%%time
scores = get_scores(errors_valid, "eval")
scores

CPU times: user 1min 31s, sys: 49.1 ms, total: 1min 31s
Wall time: 1min 34s


{'eval/tp': 0.877466251298027,
 'eval/tp_abs': 845,
 'eval/fn': 0.028037383177570093,
 'eval/fn_abs': 27,
 'eval/fp': 0.02596053997923157,
 'eval/fp_abs': 25,
 'eval/replace': 0.010384215991692628,
 'eval/replace_abs': 10,
 'eval/insert': 0.018691588785046728,
 'eval/insert_abs': 18,
 'eval/delete': 0.032191069574247146,
 'eval/delete_abs': 31,
 'eval/complex': 0.007268951194184839,
 'eval/complex_abs': 7,
 'eval/edit_distance_rel': 0.9268044884022522,
 'eval/exact_match': 0.877466251298027,
 'eval/gleu': 0.9716914437064991,
 'eval/edit_distance_abs': 1.6666666666666667}

In [47]:
errors_valid.error_type.value_counts()

tp         845
delete      31
fn          27
fp          25
insert      18
replace     10
complex      7
Name: error_type, dtype: int64

In [89]:
s = errors_valid[errors_valid.error_type != "tp"].sample(20)
show_errors(s)

0;21_hodentumoren_0269.tsv;32

__Input:__

Im Stadium cSI treten unter Überwachung, nach Carboplatin- oder operativer Therapie Rezidive zumeist retroperitoneal auf, nach einer Strahlentherapie außerhalb („ex field“) der bestrahlten Lymphabflussregion.

__Error type:__ replace

1;26_palliativmedizin_0686.tsv;22

...

__Error type:__ fp

19;08_aktinische-keratose-und-plattenepithelkarzinom-der-haut_0138.tsv;4

__Input:__

In einer 2010 publizierten, doppelt-verblindeten, Placebo-kontrollierten, randomisierten Studie wurden 32 Organtransplantierte Patienten (18 Nieren-, 8 Herz und 6 Lebertransplantierte) entweder mit Diclofenac-Natrium 3% in Hyaluronsäure 2,5% Gel oder nur Vehikel-Gel behandelt [REF].

__Error type:__ delete

In [49]:
%%time
out_test = pipeline(list(test_df.raw_sentence))

CPU times: user 8min 8s, sys: 167 ms, total: 8min 8s
Wall time: 8min 8s


In [53]:
errors_test = calculate_errors(out_test, test_df)
errors_test.error_type.value_counts()

tp         891
fn          35
delete      25
fp          23
insert      16
complex     14
replace     13
Name: error_type, dtype: int64

In [52]:
%%time
scores = get_scores(errors_test, "test")
scores

CPU times: user 1min 38s, sys: 61.1 ms, total: 1min 38s
Wall time: 1min 41s


{'test/tp': 0.8761061946902655,
 'test/tp_abs': 891,
 'test/fn': 0.0344149459193707,
 'test/fn_abs': 35,
 'test/fp': 0.02261553588987217,
 'test/fp_abs': 23,
 'test/replace': 0.012782694198623401,
 'test/replace_abs': 13,
 'test/insert': 0.015732546705998034,
 'test/insert_abs': 16,
 'test/delete': 0.024582104228121928,
 'test/delete_abs': 25,
 'test/complex': 0.01376597836774828,
 'test/complex_abs': 14,
 'test/edit_distance_rel': 0.9214101234907869,
 'test/exact_match': 0.8761061946902655,
 'test/gleu': 0.9637818113772455,
 'test/edit_distance_abs': 2.0}

## Clean up

In [ ]:
%%time
out_train = pipeline(list(train_df.raw_sentence))

errors_train = calculate_errors(out_train, train_df)

errors_train.error_type.value_counts()

In [ ]:
errors_all = pd.concat([errors_train, errors_valid, errors_test])

In [ ]:
errors_all.error_type.value_counts()

In [ ]:
show_errors(errors_all[errors_all.error_type != "tp"].sort_values(['error_type', 'file', 'sentence_id']))